[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/NBELab/afterimage/blob/main/notebooks/run.ipynb)


In [6]:
import sys
from pathlib import Path

repo = Path('afterimage').resolve()
if not repo.exists():
  !git clone https://github.com/NBELab/afterimage.git

sys.path.insert(0, str(repo))

In [ ]:
!pip install -r afterimage/requirements.txt

In [ ]:
import pyopencl

FOUND_CL = False
try:
  pyopencl.get_platforms()
  FOUND_CL = True
except Exception as e:
  print(e)

print(f'{FOUND_CL=}')


In [ ]:
from pprint import pprint
from pathlib import Path
import numpy as np

from caifi.run import run, Cfg

# root dir to save results
save_dir = Path.cwd() / 'results'
run_kwargs = dict(ocl=FOUND_CL, save_dir=save_dir, probe=False, override_dir=True)


# 1. Closed contours stimuli

In [ ]:
from caifi.stimuli.contours import CONT_COLORS, OPPOSITE_COLORS_BY_NAME, create_contour_inputs

pprint(CONT_COLORS)

h, w = 36, 36
blob_kwargs = dict(base_radius=8, wobble_ampl=3, n_wobbles=6, thickness=2)

bg_color = CONT_COLORS['bg']
inner_color = 'eq'
cont_color = CONT_COLORS['after_contour']

colors = ['blue', 'orange', 'green', 'pink']
modes = ['pos', 'neg', 'double_opp', 'double_same', 'posneg', 'negpos']
for color_name in colors:
  color = CONT_COLORS[color_name]
  for mode in modes:
    opp_color = CONT_COLORS[OPPOSITE_COLORS_BY_NAME[color_name]] if mode == 'double_opp' else None
    imgs, info = create_contour_inputs(h, w, mode=mode, color=color, bg_color=bg_color, contour_color=cont_color, inner_color=inner_color, opp_color=opp_color, **blob_kwargs)
    t_present = 1
    if mode in ['posneg', 'negpos']:
      t_present = 0.5
      imgs = np.stack([imgs[0], imgs[0], imgs[1], imgs[2]])
    cfg = Cfg(extra=info)

    exp_name = f'{color_name}-{mode}'
    run(cfg, imgs / 255, t_present=t_present, exp_name=exp_name, **run_kwargs)

#2. Star-like stimuli

In [ ]:
import functools
from caifi.stimuli.stars import create_round_stars, STAR_COLORS

pprint(STAR_COLORS)

color_kwargs = dict(color1=STAR_COLORS['red'], color2=STAR_COLORS['green'], inner_color=STAR_COLORS['center'], bg_color=STAR_COLORS['bg'], contour_color=(128, 128, 128))
create_stars = functools.partial(create_round_stars, **color_kwargs, contour_width=2, size=50, base_radius=15, wobble_ampl=5)

for inner_contour in [False, True]:
  (img, c1, c2), info = create_stars(inner_contour=inner_contour)
  cfg = Cfg(extra=info)
  desc = '-inner' if inner_contour else ''
  run(cfg, np.stack([img, c1])/255,  exp_name=f'stars-c1{desc}', t_present=1, **run_kwargs)
  run(cfg, np.stack([img, c2])/255,  exp_name=f'stars-c2{desc}', t_present=1, **run_kwargs)

# alternating
(img, c1, c2), info = create_stars(inner_contour=False)
run(cfg, np.stack([img, img, c1, c2])/255, exp_name='stars-c1c2', t_present=0.5, **run_kwargs)
run(cfg, np.stack([img, img, c2, c1])/255, exp_name='stars-c2c1', t_present=0.5, **run_kwargs)

#3. Spiral stimuli

In [ ]:
from caifi.stimuli.spiral import load_spiral

# A red spiral on a light background
(img, cfull, cout, cin), info = load_spiral(color=(250, 0, 0), bg_color=230, cont_color=10, cont_bg_color=230)
cfg = Cfg(extra=info)
run(cfg, np.stack([img, cfull])/255, exp_name='spiral-cfull', t_present=1, **run_kwargs)
run(cfg, np.stack([img, cout])/255, exp_name='spiral-cout', t_present=1, **run_kwargs)
run(cfg, np.stack([img, cin])/255, exp_name='spiral-cin', t_present=1, **run_kwargs)

# A light spiral on a red background
(img, cfull, cout, cin), info = load_spiral(color=(230, 230, 230), bg_color=(250, 0, 0), cont_color=10, cont_bg_color=230)
cfg = Cfg(extra=info)
run(cfg, np.stack([img, cfull])/255, exp_name='spiral-cfull-inv', t_present=1, **run_kwargs)
run(cfg, np.stack([img, cout])/255, exp_name='spiral-cout-inv', t_present=1, **run_kwargs)
run(cfg, np.stack([img, cin])/255, exp_name='spiral-cin-inv', t_present=1, **run_kwargs)